# DICOM to BIDS converter

Welcome to this DICOM to BIDS converter!

This converter only handles the following MRI sequences:
    - MPRAGE
    - FLAIR
    - EPI
    - Phase
    - Diffusion weighted imaging

The converter should be used on dicom files coming directly out of the MR machine.

#### /!\ Important /!\ : Run steps 1 and 2 before running anything else

Author: Maxence Wynen -- maxencewynen@gmail.com

## 1. Import dependencies

In [ ]:
import os,sys,inspect
current_dir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parent_dir = os.path.dirname(current_dir)
sys.path.insert(0, parent_dir) 
from dicom2bids import *
print("[INFO] Imports done")

## 2. Set paths

In [ ]:
# Path to dicom directory
directory = "D:/WSBIM/ROSE"
# Output BIDS directory. Should previously be created.
bids_dir = "C:/Users/maxen/OneDrive/Bureau/UCLouvain/Q14/WSBIM2243/project/WSBIM2243/database"
# Path to dcm2niix.exe converter -- Only for windows(Comment for unix/macOS users) 
dicom2niix_path = "dicom2niix" 
dicom2niix_path = "C:/Users/maxen/OneDrive/Bureau/UCLouvain/Q14/WSBIM2243/project/dcm2niix.exe" # Uncomment for Windows

## 3. Automatically import a DICOM directory into a BIDS format

In [ ]:
# Convert all DICOMs
dicom_series = convert_all_dicoms(directory, dicom2niix_path)

# Create directories in the BIDS file structure by giving an incremental id
pat_id, session = make_directories(bids_dir,pat_id=None,session='02')
# To specify the patient id:
# pat_id, session = make_directories(bids_dir,pat_id='ID_TO_SPECIFY',session=None)
# To specify the patient id and session:
# pat_id, session = make_directories(bids_dir,pat_id='ID_TO_SPECIFY',session='SESSION_TO_SPECIFY')

# Rename and move all (interesting) converted files into the bids directory
rename_and_move_nifti(dicom_series, bids_dir, pat_id, session)

print("[INFO] Done")

## 4.Optional functionalities

### 4.1. Delete subject

In [ ]:
# Remove a patient from the database
delete_subject(bids_dir, "002")

### 4.2. Delete session

In [ ]:
# Remove a particular session from a particular patient
delete_session(bids_dir, "002", "02")